# 🧪 PoC (Alumno) — AuraDB + OpenAI (NL→Cypher) con **mínimo preprocesamiento**
**Meta:** experimentar qué devuelve el modelo con un pipeline **muy básico** y reflexionar por qué hacen falta:
- **limpieza de fences** (```cypher …```),
- **bloqueo de escrituras** (CREATE/MERGE/SET/DELETE/LOAD CSV…),
- **validación** antes de ejecutar.

> Este cuaderno no incluye soluciones. Solo **pistas** y celdas con **TODO**.
**Última actualización:** 2025-09-24 22:04 UTC

## ⚠️ Aviso
- Usaremos `DRY_RUN = True` para **NO ejecutar** lo que devuelva el LLM.
- No cambies esto salvo que el profe lo indique explícitamente y el usuario sea **solo-lectura**.

## 1) Instalación rápida

In [ ]:
# TODO: instala dependencias (usa %pip)
# Pista: python-dotenv, neo4j, langchain, langchain-community, langchain-openai, tiktoken
# %pip install -q ...

## 2) Configuración (mínima)
Objetivo: cargar `.env`, crear **llm** (ChatOpenAI) y **graph** (Neo4jGraph).

**Pistas**
- `from dotenv import load_dotenv`; `load_dotenv()`
- Variables: `OPENAI_API_KEY`, `OPENAI_MODEL`, `NEO4J_URI`, `NEO4J_USERNAME`, `NEO4J_PASSWORD`
- `from langchain_openai import ChatOpenAI` + `os.environ["OPENAI_API_KEY"] = ...`
- `from langchain_community.graphs import Neo4jGraph`

In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY") or "<tu-openai-key>"
OPENAI_MODEL   = os.getenv("OPENAI_MODEL")   or "gpt-4.1-mini"
NEO4J_URI      = os.getenv("NEO4J_URI")      or "neo4j+s://<tu-host>.databases.neo4j.io"
NEO4J_USERNAME = os.getenv("NEO4J_USERNAME") or "neo4j"
NEO4J_PASSWORD = os.getenv("NEO4J_PASSWORD") or "<tu-contraseña>"

from langchain_openai import ChatOpenAI
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY
llm = ChatOpenAI(model=OPENAI_MODEL, temperature=0)

from langchain_community.graphs import Neo4jGraph
graph = Neo4jGraph(url=NEO4J_URI, username=NEO4J_USERNAME, password=NEO4J_PASSWORD)

DRY_RUN = True
print("DRY_RUN =", DRY_RUN)

/var/folders/w8/qlbrzr6d0q92wypgx2t_5vpw0000gn/T/ipykernel_10875/2927984657.py:16: LangChainDeprecationWarning: The class `Neo4jGraph` was deprecated in LangChain 0.3.8 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-neo4j package and should be used instead. To use it run `pip install -U :class:`~langchain-neo4j` and import as `from :class:`~langchain_neo4j import Neo4jGraph``.
  graph = Neo4jGraph(url=NEO4J_URI, username=NEO4J_USERNAME, password=NEO4J_PASSWORD)


DRY_RUN = True


## 3) Esquema del grafo (solo referencia)
**Pista:** `graph.refresh_schema()` y muestra un prefijo (`[:1200]`) de `graph.schema`.

In [2]:
graph.refresh_schema()
print(graph.schema[:1200] + ("..." if len(graph.schema) > 1200 else ""))

Node properties:
Movie {movieId: STRING, runtimeMin: INTEGER, grossUSD: FLOAT, year: INTEGER, description: STRING, imdbRating: FLOAT, votes: INTEGER, metascore: INTEGER, title: STRING}
Year {value: INTEGER}
Decade {value: INTEGER}
RatingBand {name: STRING, min: FLOAT, max: FLOAT}
RuntimeBand {name: STRING, min: INTEGER, max: INTEGER}
BoxOfficeBand {name: STRING, min: FLOAT, max: FLOAT}
Keyword {name: STRING}
Relationship properties:

The relationships:
(:Movie)-[:RELEASED_IN]->(:Year)
(:Movie)-[:HAS_RATING_BAND]->(:RatingBand)
(:Movie)-[:HAS_RUNTIME_BAND]->(:RuntimeBand)
(:Movie)-[:HAS_BOXOFFICE_BAND]->(:BoxOfficeBand)
(:Movie)-[:HAS_KEYWORD]->(:Keyword)
(:Year)-[:IN_DECADE]->(:Decade)


## 4) Generación **naive** de Cypher
Propósito: **no** prohibir escrituras ni limpiar fences para ver qué pasa.

**Pistas**
- Crea un `PromptTemplate` con `input_variables=["schema","question"]`
- Template: "Genera una consulta Cypher ... Esquema: {schema} ... Pregunta: {question} ... Cypher:"
- Función `naive_generate_cypher(question)` que invoque el LLM con el prompt.

In [3]:
from langchain.prompts import PromptTemplate

NAIVE_PROMPT = PromptTemplate(
    input_variables=["schema","question"],
    template=(
        "Genera una consulta Cypher para Neo4j que responda a la pregunta.\n"
        "Esquema (para contexto):\n{schema}\n\n"
        "Pregunta: {question}\n"
        "Cypher:"
    )
)

def naive_generate_cypher(question: str) -> str:
    prompt = NAIVE_PROMPT.format(schema=graph.schema, question=question)
    return llm.invoke(prompt).content

## 5) Ejecutor **naive** (imprime y, si se decide, ejecuta)
Por defecto, no ejecutaremos (DRY_RUN=True). Solo **imprime** el Cypher devuelto.

**Pistas**
- `naive_run(q)` debe:
  1) Obtener texto con `naive_generate_cypher(q)`
  2) Imprimirlo tal cual (posibles ``` o comandos peligrosos)
  3) Si `DRY_RUN` → devolver un dict con el texto y aviso
  4) Si no, ejecutar `graph.query(cy_text)` en `try/except`

In [4]:
def naive_run(question: str):
    cy_text = naive_generate_cypher(question)
    print("— Cypher devuelto (sin limpiar) —")
    print(cy_text)
    print("————————")
    if DRY_RUN:
        return {"cypher_raw": cy_text, "result": "[DRY_RUN activo: no ejecutado]"}
    try:
        result = graph.query(cy_text)
    except Exception as e:
        result = f"[ERROR al ejecutar] {type(e).__name__}: {e}"
    return {"cypher_raw": cy_text, "result": result}

## 6) Demostración A — Caso **benigno** (lectura)
Pide algo sencillo, por ejemplo: *"Dame el top 5 de películas por rating."*

**Objetivo**
- Ver si el modelo devuelve una consulta limpia o con fences (```).
- Observar diferencias de estilo/estructura.

In [5]:
out_a = naive_run("Dame el top 5 de películas por rating.")
out_a

— Cypher devuelto (sin limpiar) —
```cypher
MATCH (m:Movie)
RETURN m.title, m.imdbRating
ORDER BY m.imdbRating DESC
LIMIT 5
```
————————


{'cypher_raw': '```cypher\nMATCH (m:Movie)\nRETURN m.title, m.imdbRating\nORDER BY m.imdbRating DESC\nLIMIT 5\n```',
 'result': '[DRY_RUN activo: no ejecutado]'}

## 7) Demostración B — Caso **peligroso** (escritura)
Pide algo explícitamente destructivo, p. ej.: *"Crea una película de prueba..."*

**Objetivo**
- Comprobar si el LLM devuelve `CREATE/MERGE/SET/DELETE/...`.
- Entender por qué hay que **bloquear escrituras** antes de ejecutar.

## 8) Demostración C — Problema de fences (```cypher)
Muchos modelos envían la respuesta en un bloque con ``` que **rompe** la ejecución directa.

**Objetivo**
- Observar el problema y razonar cómo limpiarlo mínimamente.

In [ ]:
# TODO: ejecuta naive_run con otra pregunta de conteo (COUNT) y observa el formato
# out_c = naive_run("...")
# out_c

## 9) Alternativa **mínima** más segura (solo lectura + limpieza superficial)
**Sin dar la solución**, diseña dos funciones:
1) `strip_fences(text)` — Elimina ``` al inicio/fin y descarta la etiqueta inicial (p. ej., `cypher`).
   - Pista: `.strip()`, `.startswith("```")`, `.splitlines()`
2) `minimally_safe_generate(question)` — Usa tu naive, limpia con `strip_fences`, y **si** detectas palabras prohibidas (`CREATE|MERGE|SET|DELETE|REMOVE|DROP|LOAD CSV`), **lanza error** para proteger la BD.
3) `minimally_safe_run(question)` — Igual que `naive_run`, pero usando la versión “limpia y solo-lectura”.

In [6]:
import re

WRITE_RE = re.compile(r"\b(CREATE|MERGE|SET|DELETE|REMOVE|DROP|LOAD\s+CSV)\b", re.IGNORECASE)

def strip_fences(text: str) -> str:
    t = (text or "").strip()
    if t.startswith("```"):
        t = t.strip("`")
        lines = t.splitlines()
        if lines and not lines[0].strip().upper().startswith(("MATCH","CALL","RETURN","WITH")):
            lines = lines[1:]
        t = "\n".join(lines).strip()
    return t

def minimally_safe_generate(question: str) -> str:
    raw = naive_generate_cypher(question)
    cleaned = strip_fences(raw)
    if WRITE_RE.search(cleaned):
        raise ValueError("⚠️ Consulta potencialmente destructiva detectada. Bloqueada para proteger la BD.\n" + cleaned)
    return cleaned

def minimally_safe_run(question: str):
    cy = minimally_safe_generate(question)
    print("— Cypher (limpio y solo-lectura) —")
    print(cy)
    print("————————")
    if DRY_RUN:
        return {"cypher": cy, "result": "[DRY_RUN activo: no ejecutado]"}
    try:
        result = graph.query(cy)
    except Exception as e:
        result = f"[ERROR al ejecutar] {type(e).__name__}: {e}"
    return {"cypher": cy, "result": result}

## 10) Comparativa — naive vs. mínimo seguro
Usa **la misma pregunta** (p. ej., con keywords) y compara:
- ¿Hay diferencias en el Cypher final?
- ¿La versión segura bloquea algo? ¿Qué y por qué?

In [7]:
print("### NAIVE")
naive = naive_run("Dame 10 películas con 'prison' entre las keywords, ordenadas por rating.")
print("\n### MIN SAFE")
safe = minimally_safe_run("Dame 10 películas con 'prison' entre las keywords, ordenadas por rating.")
{"naive": naive, "safe": safe}

### NAIVE
— Cypher devuelto (sin limpiar) —
```cypher
MATCH (m:Movie)-[:HAS_KEYWORD]->(k:Keyword {name: 'prison'})
RETURN m
ORDER BY m.imdbRating DESC
LIMIT 10
```
————————

### MIN SAFE
— Cypher (limpio y solo-lectura) —
MATCH (m:Movie)-[:HAS_KEYWORD]->(k:Keyword {name: 'prison'})
RETURN m
ORDER BY m.imdbRating DESC
LIMIT 10
————————


{'naive': {'cypher_raw': "```cypher\nMATCH (m:Movie)-[:HAS_KEYWORD]->(k:Keyword {name: 'prison'})\nRETURN m\nORDER BY m.imdbRating DESC\nLIMIT 10\n```",
  'result': '[DRY_RUN activo: no ejecutado]'},
 'safe': {'cypher': "MATCH (m:Movie)-[:HAS_KEYWORD]->(k:Keyword {name: 'prison'})\nRETURN m\nORDER BY m.imdbRating DESC\nLIMIT 10",
  'result': '[DRY_RUN activo: no ejecutado]'}}

## 12) Conclusiones
- El LLM puede devolver **fences** y **escrituras** si no lo guiamos.
- Con dos pasos mínimos (quitar fences + bloquear escrituras) se evitan muchos problemas.
- En el cuaderno “definitivo” se añaden más protecciones (prompts estrictos, validadores, resumen controlado).

## 13) Documentación
- AuraDB — conectar apps: https://neo4j.com/docs/aura/connecting-applications/overview/
- Neo4j Python Driver: https://neo4j.com/docs/python-manual/current/
- LangChain (Neo4j): https://python.langchain.com/docs/integrations/graphs/neo4j
- Prompt Templates: https://python.langchain.com/docs/guides/prompt_templates/